In [1]:
import numpy as np
import pandas as pd
import pulp
from munkres import Munkres, DISALLOWED

In [2]:
m = Munkres()
dist_sum = 0

In [3]:
df = pd.read_csv("../kommi/ftv33.atsp", header=None, skiprows=2, engine='python')
df = df.to_numpy()
np.fill_diagonal(df, DISALLOWED)
df = df.tolist()

In [6]:
np.array(df)

array([[-99999,     26,     82, ...,    162,     62,     66],
       [    66, -99999,     56, ...,    189,    128,     40],
       [    43,     57, -99999, ...,    205,    105,     62],
       ...,
       [   146,    172,    228, ..., -99999,    124,    212],
       [   102,    128,    184, ...,    228, -99999,    168],
       [    81,     95,     38, ...,    243,    143, -99999]])

In [3]:
#diag_max = int(1e8)
df = np.zeros((10, 10), dtype = int)
df[1, 0] = 32
df[2, :2] = np.array([41, 22])
df[3, :3] = np.array([22, 50, 63])
df[4, :4] = np.array([20, 42, 41, 36])
df[5, :5] = np.array([57, 51, 30, 78, 45])
df[6, :6] = np.array([54, 61, 45, 72, 36, 22])
df[7, :7] = np.array([32, 20, 10, 54, 32, 32, 41])
df[8, :8] = np.array([22, 54, 60, 20, 22, 67, 57, 50])
df[9, :9] = np.array([45, 51, 36, 64, 28, 20, 10, 32, 50])

df = df + df.T
np.fill_diagonal(df, DISALLOWED)
df = df.tolist()

In [4]:
indexes = m.compute(df)
total_cost = 0
for r, c in indexes:
    x = df[r][c]
    total_cost += x
    print(('(%d, %d) -> %d' % (r, c, x)))
print(('lowest cost=%d' % total_cost))
dist_sum += total_cost

(0, 4) -> 20
(1, 7) -> 20
(2, 1) -> 22
(3, 8) -> 20
(4, 0) -> 20
(5, 9) -> 20
(6, 5) -> 22
(7, 2) -> 10
(8, 3) -> 20
(9, 6) -> 10
lowest cost=184


In [5]:
np.array(m.C).T

array([[-99999,     12,     31,      2,      0,     37,     44,     24,
             2,     37],
       [     0, -99999,      0,     18,     10,     19,     39,      0,
            22,     31],
       [    19,      0, -99999,     41,     19,      8,     33,      0,
            38,     26],
       [     2,     30,     53, -99999,     16,     58,     62,     46,
             0,     56],
       [     0,     22,     31,     16, -99999,     25,     26,     24,
             2,     20],
       [    25,     19,      8,     46,     13, -99999,      0,     12,
            35,      0],
       [    32,     39,     33,     50,     14,      0, -99999,     31,
            35,      0],
       [    12,      0,      0,     34,     12,     12,     31, -99999,
            30,     24],
       [     2,     34,     50,      0,      2,     47,     47,     42,
        -99999,     42],
       [    25,     31,     26,     44,      8,      0,      0,     24,
            30, -99999]])

In [6]:
def getSets(df, indexes):
    subsets = []
    isub = 0
    N = len(df)
    idx = np.zeros(N, dtype=int)
    for r, c in indexes:
        idx[r] = c

    count = 0
    while count < N:
        for i in range(N):
            if idx[i] >= 0:
                break
        subsets.append([])
        while idx[i] >= 0:
            subsets[isub].append(i)
            iold = i
            i = idx[i]
            idx[iold] = -1
            count += 1
        isub += 1
    return subsets

In [7]:
Sets0 = getSets(df, indexes)
Sets0

[[0, 4], [1, 7, 2], [3, 8], [5, 9, 6]]

In [8]:
def min_distance(matrix_prev, loop_1, loop_2):
    min_dist = matrix_prev[loop_1[0]][loop_2[0]]
    for vertex_1 in loop_1:
        for vertex_2 in loop_2:
            min_dist = min(min_dist, matrix_prev[vertex_1][vertex_2])
    return min_dist

In [9]:
#C_ = np.copy(ass_by_Hun.b).flatten()
n1 = len(Sets0)
C1 = np.zeros((n1, n1), dtype=int).tolist()
for i in range(n1):
    for j in range(n1):
        if i == j:
            C1[i][j] = DISALLOWED
            continue
        C1[i][j] = min_distance(m.C, Sets0[i], Sets0[j])

In [10]:
C1

[[-99999, 0, 2, 8],
 [12, -99999, 30, 8],
 [2, 18, -99999, 30],
 [20, 8, 42, -99999]]

In [11]:
def mindist(dist_matrix, dim, i, j):
    cur_min = np.inf
    for k in range(dim):
        if k != j and k != i:
            cur_min = min(cur_min, dist_matrix[i][k] + dist_matrix[k][j])
    return cur_min

In [11]:
def compression(dist_matrix, dim):
    for i in range(dim):
        for j in range(dim):
            if i != j:
                cur_mindist = mindist(dist_matrix, dim, i, j)
                if dist_matrix[i][j] > cur_mindist:
                    dist_matrix[i][j] = cur_mindist
                    return True, dist_matrix

    return False, dist_matrix

In [27]:
# вариант с компресссией за один проход
def compression(dist_matrix, dim):
    flag = False
    for i in range(dim):
        for j in range(dim):
            if i != j:
                cur_mindist = mindist(dist_matrix, dim, i, j)
                if dist_matrix[i][j] > cur_mindist:
                    dist_matrix[i][j] = cur_mindist
                    flag = True #, dist_matrix

    return flag, dist_matrix

In [13]:
cmprs_status = True
before_cmprs = C1.copy()
while (cmprs_status):
    print("compression", cmprs_status)
    cmprs_status, C1 = compression(C1, n1)

compression True
compression True


In [14]:
np.array(C1).T

array([[-99999,     12,      2,     20],
       [     0, -99999,      2,      8],
       [     2,     14, -99999,     22],
       [     8,      8,     10, -99999]])

In [15]:
indexes = m.compute(C1)
total_cost = 0
for r, c in indexes:
    x = C1[r][c]
    total_cost += x
    print(('(%d, %d) -> %d' % (r, c, x)))
print(('lowest cost=%d' % total_cost))
dist_sum += total_cost

(0, 2) -> 2
(1, 3) -> 8
(2, 0) -> 2
(3, 1) -> 8
lowest cost=20


In [16]:
np.array(m.C).T

array([[-99999,      4,      0,     12],
       [     0, -99999,      0,      0],
       [     0,      4, -99999,     12],
       [     8,      0,      8, -99999]])

In [17]:
Sets1 = getSets(C1, indexes)
Sets1

[[0, 2], [1, 3]]

In [18]:
#C_ = np.copy(ass_by_Hun.b).flatten()
n2 = len(Sets1)
C2 = np.zeros((n2, n2), dtype=int).tolist()
for i in range(n2):
    for j in range(n2):
        if i == j:
            C2[i][j] = DISALLOWED
            continue
        C2[i][j] = min_distance(m.C, Sets1[i], Sets1[j])

In [19]:
cmprs_status = True
before_cmprs = C2.copy()
while (cmprs_status):
    print("compression", cmprs_status)
    cmprs_status, C2 = compression(C2, n2)

compression True


In [20]:
C2

[[-99999, 0], [4, -99999]]

In [21]:
dist_sum

204

In [22]:
indexes = m.compute(C2)
total_cost = 0
for r, c in indexes:
    x = C2[r][c]
    total_cost += x
    print(('(%d, %d) -> %d' % (r, c, x)))
print(('lowest cost=%d' % total_cost))

(0, 1) -> 0
(1, 0) -> 4
lowest cost=4


In [23]:
Sets2 = getSets(C2, indexes)
Sets2

[[0, 1]]

In [24]:
#C_ = np.copy(ass_by_Hun.b).flatten()
n3 = len(Sets2)
C3 = np.zeros((n3, n3), dtype=int).tolist()
for i in range(n3):
    for j in range(n3):
        if i == j:
            C3[i][j] = DISALLOWED
            continue
        C3[i][j] = min_distance(C2, Sets2[i], Sets2[j])

In [25]:
cmprs_status = True
before_cmprs = C3.copy()
while (cmprs_status):
    print("compression", cmprs_status)
    cmprs_status, C3 = compression(C3, n3)

compression True


In [26]:
C3

[[-99999]]

In [ ]:
indexes = m.compute(C3)
total_cost = 0
for r, c in indexes:
    x = C3[r][c]
    total_cost += x
    print(('(%d, %d) -> %d' % (r, c, x)))
print(('lowest cost=%d' % total_cost))

# current solution: 1207

# optimal solution of ftv33: 1286